In [1]:
pip install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.12.0
    Uninstalling accelerate-0.12.0:
      Successfully uninstalled accelerate-0.12.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [3]:
import tensorflow as tf
from tensorflow import keras

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
#model.to(device)

In [5]:
tf.test.is_built_with_cuda()

True

In [2]:
df = pd.read_excel('nonull_reviews.xlsx')
df['msg_content']=df['row_info']

In [3]:
# Read IT texts
df2=pd.read_csv('NaturalLanguageProcessing.csv',dtype=str)
df3=pd.read_csv('YandexTeam.csv',dtype=str)
df4=pd.read_csv('LEFTJOIN.csv',dtype=str)
df5=pd.read_csv('стойподстрелой.csv',dtype=str)
df6=pd.read_csv('эйайньюз_text.csv',dtype=str)
df7=pd.read_csv('наташинчат.csv',dtype=str)

In [4]:
# Concat them
frames=[df,df2,df3,df4,df5,df6,df7]
df_it=pd.concat(frames)

In [5]:
df_it

,Unnamed: 0,date,status,whois,position,average,recommended,row_info,company_name,employees,text_len,msg_content,msg_id
0,0.0,17 октября 2018,Бывший сотрудник,Стаж в компании: от 1 года до 3 лет. Пенза.,Инженер по автоматизации тестирования,Оценка1.3,Рекомендация10%,Можно идти на год только если вы junior и нику...,Bell Integrator,От 1000 до 5000 человек,264.0,Можно идти на год только если вы junior и нику...,NaN
1,1.0,25 июля 2018,Бывший сотрудник,Стаж в компании: от 3 месяцев до 1 года. Москва.,Фронтенд разработчик,Оценка2.2,Рекомендация20%,Достоинства При большом желании можно хорошо р...,Bell Integrator,От 1000 до 5000 человек,242.0,Достоинства При большом желании можно хорошо р...,NaN
2,2.0,24 июля 2018,Текущий сотрудник,Стаж в компании: от 3 месяцев до 1 года. Москва.,Системный администратор,Оценка2.9,Рекомендация70%,Достоинства Достойная ЗП Недостатки Хаотичный ...,Bell Integrator,От 1000 до 5000 человек,162.0,Достоинства Достойная ЗП Недостатки Хаотичный ...,NaN
3,3.0,4 июля 2018,Бывший сотрудник,NaN,Инженер по ручному тестированию,Оценка2.6,Рекомендация0%,Компания подойдет джунам для старта карьеры и ...,Bell Integrator,От 1000 до 5000 человек,376.0,Компания подойдет джунам для старта карьеры и ...,NaN
4,4.0,23 мая 2018,Бывший сотрудник,NaN,NaN,Оценка3.4,Рекомендация60%,проекты на которых задействованы сотрудники - ...,Bell Integrator,От 1000 до 5000 человек,171.0,проекты на которых задействованы сотрудники - ...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
326576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Ну в целом да, все просто красят кнопки",367954
326577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""start"" ""end"" ""promise1"" ""timer1"" ""promise2""",367955
326578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"хмм точно так же можно аргументировать то, поч...",367956
326579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,все большинство,367957


In [7]:
# Delete excess symbols etc

df_it['msg_content'] = df_it['msg_content'].replace(r'[@#$%^&*()]|http\S+', '', regex=True).replace(r'[\u2000-\u3300]|[\U0001F600-\U0001F64F]|[\U0001F300-\U0001F5FF]|[\U0001F680-\U0001F6FF]|[\U0001F1E0-\U0001F1FF]', '', regex=True).replace(r'\d+', '', regex=True)

In [8]:
# Import rubert-tiny2

from transformers import BertTokenizer, BertForMaskedLM
import torch

tokenizer = BertTokenizer.from_pretrained('cointegrated/rubert-tiny2')
model = BertForMaskedLM.from_pretrained('cointegrated/rubert-tiny2')

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/401 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/715 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
text=df_it['msg_content'].tolist()

In [12]:
# Raises error ValueError: "Input is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers."
# locally but fine witihin kaggle notebook
inputs = tokenizer(text, return_tensors='pt', max_length=256, truncation=True, padding='max_length')

ValueError: Input is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.

In [9]:
# Try if all works with sample and replace inputs with sample_inputs
#text_sample=text[:2000]
#sample_inputs= tokenizer(text_sample, return_tensors='pt', max_length=256, truncation=True, padding='max_length')

In [17]:
inputs['labels'] = inputs.input_ids.detach().clone()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_23/827236631.py:1 in <module>                                                     │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_23/827236631.py'                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'inputs' is not defined

In [ ]:
inputs.keys()

In [ ]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [ ]:
mask_arr

In [ ]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [ ]:
# Check
selection[:5]

In [ ]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [ ]:
# Check
inputs.input_ids

In [ ]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = MeditationsDataset(inputs)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

In [ ]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

In [ ]:
# Takes 12 mins for 15k texts and about 7hrs for 520k texts???
from tqdm import tqdm  # for our progress bar

epochs = 2

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

In [ ]:
model.save_pretrained("CustomModels/CustomMLM_BIG")
tokenizer.save_pretrained("CustomModels/CustomMLM_BIG")